In [ ]:
t1=time.time()
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 1)
off = np.zeros(len(X_ridge))
score_list = []
fold = 1
test_preds = []


for train_index, test_index in kf.split(X_ridge, y_ridge):
    
    #ilgili iloclara göre split edilmesi
    X_train , X_val = X_ridge.iloc[train_index], X_ridge.iloc[test_index]  
    y_train, y_val = y_ridge.iloc[train_index], y_ridge.iloc[test_index]    
    
    print("X_train shape is :", X_train.shape,"X_val shape is", X_val.shape)
    y_pred_list = []
    
    #*******************SCALER*******************
    scaler = StandardScaler()
    X_train_sc = pd.DataFrame(scaler.fit_transform(X_train))
    X_val_sc = pd.DataFrame(scaler.transform(X_val))
    
    #*******************RESAMPLING*******************
    sm = SMOTE(sampling_strategy=0.6)
    X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train_sc, y_train)
    
    ridge = RidgeClassifier(alpha = 0.1,
                            normalize=False, random_state=1, solver='auto')
    
    ridge.fit(X_train_oversampled, y_train_oversampled) 
    
    y_pred_list.append(ridge.predict(X_val_sc))
    
    print(" Roc Auc Score", roc_auc_score(y_val, np.mean(y_pred_list, axis=0)))
    print(confusion_matrix(y_val,ridge.predict(X_val_sc)))
    print(80  * "*")
     
        
    test_preds.append(ridge.predict(test_ridge))
    off[test_index] = np.mean(y_pred_list, axis=0)
    
    score = roc_auc_score(y_val, off[test_index])
    print(f"Auc- Fold - {fold}: {score}")
    score_list.append(score)
    

    
    # print(y_pred_list.columns)
    # print("F1_Score :", f1_score(y_val,y_pred_list))
    fold +=1

    
print(np.mean(score_list))
t2=time.time()
print("Ridge model take : {:.3f} sn.".format(t2-t1))